# Assignment 2 – Quantum State Reconstruction




## 1. Imports and Reproducibility

We fix random seeds so results are **exactly reproducible**.

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

np.random.seed(42)
torch.manual_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

## 2. Quantum Background (Important)

A **single-qubit density matrix** ρ must satisfy:
- Hermitian: ρ = ρ†  
- Positive Semi-Definite: eigenvalues ≥ 0  
- Unit Trace: Tr(ρ) = 1

### Enforcing Physics (Key Idea)
We predict a **lower triangular matrix** L:

\begin{align}
\rho = \frac{LL^\dagger}{\mathrm{Tr}(LL^\dagger)}
\end{align}

This guarantees all physical constraints automatically.

## 3. Pauli Measurement Operators

We simulate measurement data using Pauli operators {X, Y, Z}.

In [3]:
I = np.eye(2)
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])

paulis = [X, Y, Z]

## 4. Dataset Generation

We generate **random valid density matrices** and compute expectation values:

\begin{align}
\langle O \rangle = \mathrm{Tr}(\rho O)
\end{align}

In [4]:
def random_density_matrix():
    A = np.random.randn(2,2) + 1j*np.random.randn(2,2)
    rho = A @ A.conj().T
    rho /= np.trace(rho)
    return rho

def measure_expectations(rho):
    return np.array([np.real(np.trace(rho @ P)) for P in paulis])

N = 5000
X_data = []
Y_data = []

for _ in range(N):
    rho = random_density_matrix()
    X_data.append(measure_expectations(rho))
    Y_data.append(rho)

X_data = torch.tensor(X_data, dtype=torch.float32)
Y_data = torch.tensor(np.stack(Y_data), dtype=torch.complex64)

X_data.shape, Y_data.shape

C:\Users\KISHORE S\AppData\Local\Temp\ipykernel_35752\3556208810.py:19: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:256.)
  X_data = torch.tensor(X_data, dtype=torch.float32)


(torch.Size([5000, 3]), torch.Size([5000, 2, 2]))

## 5. Neural Network Model

The model outputs **4 real numbers** → parameters of lower-triangular matrix L.

\begin{align}
L = \begin{bmatrix}
a & 0 \\
b + ic & d
\end{bmatrix}
\end{align}

In [8]:
class DensityNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(3, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 4)
        )

    def forward(self, x):
        return self.net(x)

def params_to_density(params):
    a, b, c, d = params[:, 0], params[:, 1], params[:, 2], params[:, 3]

    L = torch.zeros((params.shape[0], 2, 2), dtype=torch.cfloat, device=params.device)
    L[:, 0, 0] = a
    L[:, 1, 0] = b + 1j * c
    L[:, 1, 1] = d

    rho = L @ L.conj().transpose(-1, -2)

    trace = torch.real(torch.diagonal(rho, dim1=1, dim2=2).sum(-1))
    rho = rho / trace.view(-1, 1, 1)

    return rho


## 6. Loss Function

We minimize **Mean Squared Error** between predicted and true density matrices.

In [9]:
def density_loss(rho_pred, rho_true):
    return torch.mean(torch.abs(rho_pred - rho_true)**2)

## 7. Training Loop

In [13]:
model = DensityNet().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

X_train = X_data.to(device)
Y_train = Y_data.to(device)

for epoch in range(65):
    optimizer.zero_grad()
    params = model(X_train)
    rho_pred = params_to_density(params)
    loss = density_loss(rho_pred, Y_train)
    loss.backward()
    optimizer.step()

    if epoch % 5 == 0:
        print(f"Epoch {epoch:02d} | Loss: {loss.item():.6f}")

Epoch 00 | Loss: 0.177288
Epoch 05 | Loss: 0.087579
Epoch 10 | Loss: 0.063423
Epoch 15 | Loss: 0.053954
Epoch 20 | Loss: 0.045159
Epoch 25 | Loss: 0.039128
Epoch 30 | Loss: 0.033295
Epoch 35 | Loss: 0.027523
Epoch 40 | Loss: 0.023416
Epoch 45 | Loss: 0.020079
Epoch 50 | Loss: 0.017788
Epoch 55 | Loss: 0.015916
Epoch 60 | Loss: 0.013508


## 8. Evaluation Metrics

### Fidelity
\begin{align}
F(\rho, \sigma) = \left(\mathrm{Tr}\sqrt{\sqrt{\rho}\sigma\sqrt{\rho}}\right)^2
\end{align}

### Trace Distance
\begin{align}
D(\rho, \sigma) = \frac{1}{2}||\rho-\sigma||_1
\end{align}

In [19]:
def fidelity(rho, sigma):
    """
    Uhlmann fidelity for 2x2 density matrices
    F(rho, sigma) = (Tr sqrt(sqrt(rho) sigma sqrt(rho)))^2
    Implemented via eigen-decomposition (PyTorch-safe)
    """
    # Eigen-decomposition of rho
    evals, evecs = torch.linalg.eigh(rho)

    # Clamp for numerical stability
    evals = torch.clamp(evals, min=0)

    # sqrt(rho)
    sqrt_rho = (evecs * torch.sqrt(evals)) @ evecs.conj().T

    # Middle product
    interm = sqrt_rho @ sigma @ sqrt_rho

    # Eigenvalues of intermediate matrix
    evals_inter, _ = torch.linalg.eigh(interm)
    evals_inter = torch.clamp(evals_inter, min=0)

    return torch.sum(torch.sqrt(evals_inter))**2



with torch.no_grad():
    params = model(X_train)
    rho_pred = params_to_density(params)
    F = torch.mean(torch.stack([fidelity(rho_pred[i], Y_train[i]) for i in range(100)]))

F.item()

0.9502231478691101

---
### AI Attribution (Required)
This notebook was developed with assistance from AI tools and mathematically verified by checking:
- Hermiticity
- Positive eigenvalues
- Unit trace
